In [1]:
import torch
import torch.nn as nn

from model import VQAModel

HF_HUB_DISABLE_SYMLINKS_WARNING =1 

c:\Users\Penguin\anaconda3\envs\vqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. 테스트용 하이퍼파라미터 설정
BATCH_SIZE = 4
IMG_SIZE = 224
SEQ_LEN = 12   # DataLoader 테스트 시 출력된 최대 길이 (예시)
NUM_CLASSES = 13 # DataLoader 테스트 시 출력된 고유 답변 수

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 더미 데이터 생성 (DataLoader의 출력 모방)
print("\n--- 1. 더미 데이터 생성 중... ---")

# (1) 이미지 배치
# Shape: (B, C, H, W), Type: float
dummy_images = torch.randn(BATCH_SIZE, 3, IMG_SIZE, IMG_SIZE).to(device)
print(f"Dummy Images shape: {dummy_images.shape}")

# (2) 텍스트 입력 배치 (BatchEncoding 딕셔너리 모방)
# Shape: (B, L), Type: long (BERT vocab 인덱스)
# 0(PAD), 101(CLS), 102(SEP) 외의 임의의 인덱스 사용
dummy_input_ids = torch.randint(low=1000, high=30000, 
                                size=(BATCH_SIZE, SEQ_LEN), 
                                dtype=torch.long).to(device)

# Shape: (B, L), Type: long
dummy_attention_mask = torch.ones(BATCH_SIZE, SEQ_LEN, dtype=torch.long).to(device)

# 실제 DataLoader 출력과 유사하게 딕셔너리로 묶기 (필수는 아님)
dummy_inputs = {
    'input_ids': dummy_input_ids,
    'attention_mask': dummy_attention_mask
}
print(f"Dummy 'input_ids' shape: {dummy_inputs['input_ids'].shape}")
print(f"Dummy 'attention_mask' shape: {dummy_inputs['attention_mask'].shape}")

Using device: cuda

--- 1. 더미 데이터 생성 중... ---
Dummy Images shape: torch.Size([4, 3, 224, 224])
Dummy 'input_ids' shape: torch.Size([4, 12])
Dummy 'attention_mask' shape: torch.Size([4, 12])


In [3]:
# 3. 모델 초기화 및 테스트 (Attention 퓨전 사용)
print("\n--- 2. 모델('attention' 퓨전) 순전파 테스트 ---")

# ★ num_classes를 실제 값(13)으로 설정
try:
    model = VQAModel(
        fusion_type="attention", 
        num_classes=NUM_CLASSES
    ).to(device)
    
    model.train() # (BatchNorm, Dropout을 위해 .train() 모드)

    # 모델 순전파
    # 
    output = model(
        images=dummy_images, 
        input_ids=dummy_inputs['input_ids'], 
        attention_mask=dummy_inputs['attention_mask']
    )
    
    print(f"\n[성공] 'attention' 모델 출력 shape: {output.shape}")
    print(f"  (예상 shape: ({BATCH_SIZE}, {NUM_CLASSES}))")

except Exception as e:
    print(f"\n[실패] 'attention' 모델 테스트 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

# 4. 모델 초기화 및 테스트 (Concat 퓨전 사용)
print("\n--- 3. 모델('concat' 퓨전) 순전파 테스트 ---")

try:
    model_concat = VQAModel(
        fusion_type="concat", 
        num_classes=NUM_CLASSES
    ).to(device)
    
    model_concat.train()
    
    # 모델 순전파
    output_concat = model_concat(
        images=dummy_images, 
        input_ids=dummy_inputs['input_ids'], 
        attention_mask=dummy_inputs['attention_mask']
    )

    print(f"\n[성공] 'concat' 모델 출력 shape: {output_concat.shape}")
    print(f"  (예상 shape: ({BATCH_SIZE}, {NUM_CLASSES}))")
    
except Exception as e:
    print(f"\n[실패] 'concat' 모델 테스트 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()


--- 2. 모델('attention' 퓨전) 순전파 테스트 ---

[성공] 'attention' 모델 출력 shape: torch.Size([4, 13])
  (예상 shape: (4, 13))

--- 3. 모델('concat' 퓨전) 순전파 테스트 ---

[성공] 'concat' 모델 출력 shape: torch.Size([4, 13])
  (예상 shape: (4, 13))
